#Yosemite National Park - Seasons

As is true of all mountainous regions, weather in the Sierra Nevada can change rapidly at any time of the year. Elevation plays a major role in temperature and precipitation variability. Since Yosemite varies in elevation from 2,000 feet to over 13,000 feet, the lower foothills of the park can be experiencing the rebirth of spring while the higher elevations remain in the grip of winter. The higher you go, the colder and more temperamental the climate gets. March, April, October, and November are transitional months when warm, sunny days can suddenly become stormy. Although summer in the Sierra is known for its sunny days, it is not unheard of for clouds to build up suddenly, creating a downpour that could last a few hours or several days. See descriptions below for more information on each season.https://www.nps.gov/yose/planyourvisit/seasons.htm

![](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSTaZTSEcp1uo7mG990QY03lPPX30bwjNeXNA&usqp=CAU)vroomvroomvroom.com

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from colorama import Fore, Back, Style 
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import xgboost
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
from statsmodels.formula.api import ols
import plotly.graph_objs as gobj
import argparse
import cv2
import cv2 as cv
import pydicom as dicom
from pydicom.filereader import dcmread
import pydicom

In [ ]:
import re
from PIL import Image
from IPython.display import Image as show_gif
from PIL import Image
from IPython.display import Image as show_gif
import scipy.misc
import matplotlib
from skimage import exposure
import numpy as np
import os
import matplotlib.pyplot as plt
from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
import plotly.figure_factory as ff
from plotly.graph_objs import *
init_notebook_mode(connected=True)

In [ ]:
image = cv2.imread('/kaggle/input/summer2winter-yosemite/testA/2011-06-14 23_29_30.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

In [ ]:
import pydicom as dicom
from pydicom.filereader import dcmread
import pydicom

#I got no `pixel_array` therefore my Image=image

In [ ]:
#Code from Salman Faroz https://www.kaggle.com/salmanfaroz/fibrosis-let-s-talk-to-the-data/notebook

# Let's define our Kaggle Notebook size
KNotebook = np.ones((5,5), np.uint8)
#Image=image.pixel_array
Image=image
Image=exposure.equalize_adapthist(Image)
plt.figure(figsize = (65,35))
plt.axis('off')

plt.subplot(341)

# Now we erode
erosion = cv2.erode(Image, KNotebook, iterations = 1)
plt.axis('off')
plt.title("Erosion", fontsize=50)

plt.imshow(erosion)

plt.subplot(342, frameon=False)

KNotebook = np.ones((5,5), np.uint8)
dilation = cv2.dilate(Image, KNotebook, iterations = 1)
plt.axis('off')
plt.title("Dilation", fontsize=50)

plt.imshow(dilation)

plt.subplot(343, frameon=False)

# Opening - Good for removing noise
opening = cv2.morphologyEx(Image, cv2.MORPH_OPEN, KNotebook)
plt.axis('off')
plt.title("Opening", fontsize=50)

plt.imshow(opening)

plt.subplot(344, frameon=False)

# Closing - Good for removing noise
closing = cv2.morphologyEx(Image, cv2.MORPH_CLOSE, KNotebook)
plt.title("Closing", fontsize=50)
plt.axis('off')

plt.imshow(closing)

In [ ]:
def make_rainbowmask(img, display=False):
    row_size= img.shape[1]
    col_size = img.shape[1]
    
    mean = np.mean(img)
    std = np.std(img)
    img = img-mean
    img = img/std
    # Find the average pixel value near the lungs
    # to renormalize washed out images
    middle = img[int(col_size/5):int(col_size/5*4),int(row_size/5):int(row_size/5*4)] 
    mean = np.mean(middle)  
    max = np.max(img)
    min = np.min(img)
    # To improve threshold finding, I'm moving the 
    # underflow and overflow on the pixel spectrum
    img[img==max]=mean
    img[img==min]=mean
    #
    # Using Kmeans to separate foreground (soft tissue / bone) and background (lung/air)
    #
    kmeans = KMeans(n_clusters=2).fit(np.reshape(middle,[np.prod(middle.shape),1]))
    centers = sorted(kmeans.cluster_centers_.flatten())
    threshold = np.mean(centers)
    thresh_img = np.where(img<threshold,1.0,0.0)  # threshold the image

    # First erode away the finer elements, then dilate to include some of the pixels surrounding the lung.  
    # We don't want to accidentally clip the lung.

    eroded = morphology.erosion(thresh_img,np.ones([1,1]))
    dilation = morphology.dilation(eroded,np.ones([8,8]))

    labels = measure.label(dilation) # Different labels are displayed in different colors
    label_vals = np.unique(labels)
    regions = measure.regionprops(labels)
    good_labels = []
    for prop in regions:
        B = prop.bbox
        if B[2]-B[0]<row_size/10*9 and B[3]-B[1]<col_size/10*9 and B[0]>row_size/5 and B[2]<col_size/5*4:
            good_labels.append(prop.label)
    mask = np.ndarray([row_size,col_size],dtype=np.int8)
    mask[:] = 0

    #
    #  After just the lungs are left, we do another large dilation
    #  in order to fill in and out the lung mask 
    #
    for N in good_labels:
        mask = mask + np.where(labels==N,1,0)
    mask = morphology.dilation(mask,np.ones([10,10])) # one last dilation

    if (display):
        fig, ax = plt.subplots(3, 2, figsize=[12, 12])
        ax[0, 0].set_title("Original")
        ax[0, 0].imshow(img, cmap='gray')
        ax[0, 0].axis('off')
        ax[0, 1].set_title("Threshold")
        ax[0, 1].imshow(thresh_img, cmap='gray')
        ax[0, 1].axis('off')
        ax[1, 0].set_title("After Erosion and Dilation")
        ax[1, 0].imshow(dilation, cmap='gray')
        ax[1, 0].axis('off')
        ax[1, 1].set_title("Color Labels")
        ax[1, 1].imshow(labels)
        ax[1, 1].axis('off')
        ax[2, 0].set_title("Final Mask")
        ax[2, 0].imshow(mask, cmap='gray')
        ax[2, 0].axis('off')
        ax[2, 1].set_title("Apply Mask on Original")
        ax[2, 1].imshow(mask*img, cmap='gray')
        ax[2, 1].axis('off')
        
        plt.show()
    return mask*img
img = image[230]
make_rainbowmask(img, display=True)

#After 1h41 `make_rainbow` didn't work. No mask or rainbow. Other things didn't work too. Just a black square.

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#eb3434','#eb3446','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Marília Prata, @mpwolke Was here' )